# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [65]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [66]:
weatherData = pd.read_csv("cityWeatherOutput.csv")
weatherData

,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,Chui,-33.6971,-53.4616,54.50,69,100,3.53,UY,1629735059
1,Kiunga,-6.1219,141.2906,73.53,100,100,0.47,PG,1629735125
2,New Norfolk,-42.7826,147.0587,37.71,76,6,2.93,AU,1629735097
3,Esim,4.8699,-2.2405,79.93,76,72,6.69,GH,1629735125
4,Coquimbo,-29.9533,-71.3436,58.73,62,40,11.50,CL,1629735126
...,...,...,...,...,...,...,...,...,...
581,Toora-Khem,52.4675,96.1100,48.27,86,63,1.97,RU,1629735291
582,Kampene,-3.6000,26.6667,80.28,58,96,0.74,CD,1629735292
583,Semikarakorsk,47.5194,40.8112,79.97,53,53,11.97,RU,1629735292
584,Rabo de Peixe,37.8000,-25.5833,76.51,78,20,11.50,PT,1629735292


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [67]:
# Gmap config 

gmaps.configure(api_key=g_key)

# Store lat lng in locations
locations = weatherData[['Latitude', 'Longitude']]

# Store Humidity as weight
humidity = weatherData['Humidity']

In [68]:
# Plot heatmap 

heatFig = gmaps.figure()

# Create heat layer
heatLayer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity = 100, point_radius = 1)

# Add layer
heatFig.add_layer(heatLayer)
heatFig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [69]:
narrowed_df = weatherData.loc[(weatherData['Max Temperature'] < 80) & (weatherData['Max Temperature'] > 70)
                             & (weatherData['Wind Speed'] < 10) & (weatherData['Cloudiness'] == 0)].dropna()

narrowed_df.head()

,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
129,Rosh Pinna,32.9689,35.5426,79.74,49,0,7.99,IL,1629735157
157,Karratha,-20.7377,116.8463,72.07,61,0,6.04,AU,1629734874
186,Kumul,42.8000,93.4500,77.34,13,0,3.60,CN,1629735170
255,Matagami,49.7502,-77.6328,75.02,47,0,2.08,CA,1629735192
321,Willmar,45.1219,-95.0433,77.31,33,0,4.79,US,1629735213


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [70]:
# Create hotel_df and add hotel name column

hotel_df = narrowed_df

hotel_df["Hotel Name"] = ""

hotel_df

,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
129,Rosh Pinna,32.9689,35.5426,79.74,49,0,7.99,IL,1629735157,
157,Karratha,-20.7377,116.8463,72.07,61,0,6.04,AU,1629734874,
186,Kumul,42.8000,93.4500,77.34,13,0,3.60,CN,1629735170,
255,Matagami,49.7502,-77.6328,75.02,47,0,2.08,CA,1629735192,
321,Willmar,45.1219,-95.0433,77.31,33,0,4.79,US,1629735213,
340,Marquette,46.5333,-87.6335,76.82,58,0,9.80,US,1629734973,
424,Sarkand,45.4100,79.9186,70.39,29,0,5.53,KZ,1629735031,


In [71]:
baseURL = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : "5000",
          "key" : g_key}

# Iterate through the list and find the hotels 
for index, row in hotel_df.iterrows():
    lat = row['Latitude']
    lng = row['Longitude']
    city_name = row['City']
    
    params['location'] = f"{lat},{lng}"

    print(f" Retrieving Results for Index {index}: {city_name}.")
    repsonse = requests.get(baseURL, params=params).json()
    
    results = repsonse['results']
    
    # Save hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # Show missing field if no hotel available
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("---")
    
    # Wait 1 sec to make another api request to avoid SSL Error 
    # Found online - not sure signficance 
    time.sleep(1)

# Print end of search once searching is completed
print("-----End of Search-----")

 Retrieving Results for Index 129: Rosh Pinna.
Closest hotel in Rosh Pinna is Spa Hotel Mizpe Hayamim.
---
 Retrieving Results for Index 157: Karratha.
Closest hotel in Karratha is Best Western Plus The Ranges Karratha.
---
 Retrieving Results for Index 186: Kumul.
Closest hotel in Kumul is 哈密环球大酒店.
---
 Retrieving Results for Index 255: Matagami.
Closest hotel in Matagami is Hôtel Matagami.
---
 Retrieving Results for Index 321: Willmar.
Closest hotel in Willmar is Best Western Plus Willmar.
---
 Retrieving Results for Index 340: Marquette.
Closest hotel in Marquette is Jasper Ridge Inn Ishpeming.
---
 Retrieving Results for Index 424: Sarkand.
Closest hotel in Sarkand is Гостиница Сарканд.
---
-----End of Search-----


In [72]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [73]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)


# Add the layer to the map 
heatFig.add_layer(markers)
# Display figure
heatFig

Figure(layout=FigureLayout(height='420px'))